# Wrangling OpenStreetmap data with SQL
---
Welcome, sir/madam! You are reading the project submission for Project 3 of the Udacity Data Analyst nanodegree, as written by Jasper Müller. This document will hopefully answer all the questions you ever had about the cleanliness of OSM data of the beauftiful Dutch city of Rottterdam. 

## Rotterdam
This is the place where I work as a Data Scientist, and am planning to move to soon. The second largest city in the Netherlands, known for its harbour, its many cultural activities, and its relativly cheap housing. For more information, I suggest you read their [wikipedia page](https://en.wikipedia.org/wiki/Rotterdam). 

Get your copy of the OSM XML data via either one of the following links:

- https://www.openstreetmap.org/relation/1411101#map=12/51.9280/4.4906
- http://metro.teczno.com/#rotterdam


# Problems Encountered in the Map
---

## Auditing Postal Codes
The first potential issue that comes to mind is to check the formatting of postal codes. Dutch postal codes should all consist of four numbers, followed by two capital letters. There should not be a space between the numbers and the letters. Moreover, for the area of Rotterdam all postal codes should start with either a 2, a 3, or a 4. 

Auditing the data I find that indeed there are some problematic postal codes. This is mainly caused by a space between the numbers and letters, but there are some other cases: 

In [52]:
%run check_postal_codes.py

{'letters_only': 0,
 'lower': 7,
 'numbers_only': 1,
 'other': 0,
 'space': 331,
 'valid': 613827}


There are 7 postal codes that contain lower-case rather than upper-case letters, and there is even one postcode that consists solely of numbers. 

## Correcting postal codes
Correcting postal codes seems easy. I would say I use the following code to correct the codes: 

In [54]:
postal_space = '2611 AN'
postal_lower_1 = '2611eg'
postal_lower_2 = '1316 ve'

postals = [postal_space, postal_lower_1, postal_lower_2]

for postal in postals: 
    corrected = postal.replace(" ", "").upper()
    print corrected

2611AN
2611EG
1316VE


Implementing the code above to loop over the OSM data of Rotterdam, we find the statistics below. By the way, I used to live at all of these three postal codes!

In [ ]:
%run correct_postal_codes.py

### Count different tag types

In [2]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict

file_in = 'sample.osm'


def count_tags(filename):
    tag_count = defaultdict(int)

    for event, elem in ET.iterparse(filename):
        tag_name = elem.tag
        tag_count[tag_name] += 1

    return tag_count

pprint.pprint(count_tags(file_in))

defaultdict(<type 'int'>, {'node': 30026, 'nd': 38450, 'member': 829, 'tag': 52464, 'relation': 25, 'way': 4740, 'osm': 1})


I.e. we have ±30.000 nodes and ±4.700 ways. That's good to know!

### Count problematic tag types

In [3]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        key = element.attrib['k']

        if lower.search(key):
            keys['lower'] += 1
        elif lower_colon.search(key):
            keys['lower_colon'] += 1
        elif problemchars.search(key):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map(file_in)
pprint.pprint(keys)

{'lower': 19327, 'lower_colon': 33121, 'other': 16, 'problemchars': 0}


### Get unique users

In [5]:
def get_user(element):
    try:
        return element.attrib['user']
    except:
        return None


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user = get_user(element)
        if user != None:
            users.add(user)
    return users

users = process_map(file_in)
print len(users)
pprint.pprint(users)

262
set(['2hands4you',
     '3dShapes',
     'A67-A67',
     'AEelderink',
     'AND',
     'AND_fixbot',
     'A_rnold',
     'Aad van Boven',
     'Ad Koolmees',
     'AlbertP_BAG',
     'AlexDT',
     'Alkmaarder',
     'Alsoft',
     'Amaroussi',
     'AndiDelft',
     'AnkEric',
     'ArAge',
     'Artep',
     'Arthur Pijpers',
     'AxiMaxi',
     'BAGgeraar',
     'Bassniper',
     'Beaker73',
     'Bert20',
     'Biff',
     'BikePC',
     'Bob de Vreede',
     'BugBlue',
     'CJTmmr',
     'CJTmmr_BAG',
     'Capellenaar',
     'CeesW',
     'Commodoortje',
     u'Corn\xe9 Pronk',
     'CyFo',
     'DAI',
     'DWizzy',
     'Davio',
     'Delft12',
     'Dirk V',
     'Emiel1',
     'Emile',
     'EmptyNutshell',
     'Erik@Zuidplas',
     'Frank314',
     'Fred Marchee',
     'FvGordon',
     'Geogoeroe',
     'GercoKees',
     'Gert van der Neut',
     'Gertjan Idema',
     'Gijsrooy',
     'Guppy_78',
     'GuustFlater',
     'HYS',
     'Hamakers',
     'Hanno Lans',
  

Apparently we have a total of 262 unique users that contributed to this map. 

## Ideas for initial exploration of data

1. Check postal codes, should all be in format 9999AA and should start with a 2, 3, or 4 in this region. 
2. Check street and city names for weird characters such as: ,./?'";:[]{}-_+=()*&^%$#@!`~
3. Since I will be living in Rotterdam soon, I would like to check out where the most bars are. 

### Postal codes
Here is an initial script that I wrote. It checks for postal codes that do not start with a 2, 3, or 4. 


In [51]:
postal_lower = re.compile(r'[\d]{4} *[a-z]{2}')
postal_lower.search('2611an').group()

'2611an'

False